<a href="https://colab.research.google.com/github/sundriesofficial/hios/blob/main/glow_tts_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCE-TTS: Glow-TTS 학습 (v1)

이 문서는 SCE-TTS 프로젝트의 Glow-TTS 학습용 문서입니다.

이 데모에 대한 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다.  
https://sce-tts.github.io/#/v1/index

실제 학습을 진행하시려면, 위쪽 메뉴에서 `런타임 -> 모두 실행`을 클릭하신 후,  
문서 최하단의 `Glow-TTS 학습 진행`까지 정상적으로 실행되는걸 확인해주시면 됩니다.

도중에 문제가 발생한다면, 문제의 원인을 해결한 후 `런타임 -> 런타임 초기화`를 클릭하고,  
다시 `런타임 -> 모두 실행`을 클릭해주세요.

In [1]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./


fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command python setup.py egg_info
  /content/apex/setup.py:43: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
    warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")
  torch.__version__  =  2.0.1+cu118
  running egg_info
  creating /tmp/pip-pip-egg-info-waa011e2/apex.egg-info
  writing /tmp/pip-pip-egg-info-waa011e2/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-waa011e2/apex.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-waa011e2/apex

In [2]:
!pip install unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.5 MB/s eta 0:00:00


## 1. 할당된 GPU 확인

현재 런타임에 할당된 GPU를 확인합니다.

만약, `GPU: NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.` 라는 메시지가 출력된다면, 위쪽 메뉴에서 `런타임 -> 런타임 유형 변경`을 클릭하고 하드웨어 가속기를 `GPU`로 변경하여 저장한 후 다시 실행해주세요.

In [3]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla T4


## 2. 구글 드라이브 마운트

음성합성을 위해 학습한 모델이 있는 구글 드라이브를 마운트합니다.  
마운트할 구글 드라이브 내에 다음 파일이 존재하는지 꼭 확인해주세요.

- `/Colab Notebooks/data/filelists.zip`

(존재하지 않는다면, [음성 데이터셋 변환](https://sce-tts.github.io/#/v1/recoding?id=%EC%9D%8C%EC%84%B1-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EB%B3%80%ED%99%98)문서를 참조하여 준비해주세요.)

만약 아래에 `Enter your authorization code:`과 같은 메시지가 출력될 경우,  
같이 출력된 링크에 접속하여, 마운트할 구글 계정을 선택하신 후, 인증 코드를 복사하여 입력해주세요.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [5]:
# Change CUDA version to 10.1
!rm /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

# check if installed successfully
!nvcc --version

/bin/bash: nvcc: command not found


In [6]:
import sys
from pathlib import Path
from pprint import pprint

In [7]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [8]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/glow-tts.git
%cd /content/glow-tts
!pip install -q --no-cache-dir "torch==1.5.1" -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -q --no-cache-dir "cython==0.29.12" "librosa==0.6.0" "numpy==1.16.4" "scipy==1.3.0" "numba==0.48" "Unidecode==1.0.22" "tensorflow==2.3.0" "inflect==4.1.0" "matplotlib==3.3.0"

/content
fatal: destination path 'glow-tts' already exists and is not an empty directory.
/content/glow-tts
ERROR: Could not find a version that satisfies the requirement torch==1.5.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.5.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [9]:
%cd /content
!git clone https://github.com/NVIDIA/apex /content/apex
%cd /content/apex
!git checkout 37cdaf4
!pip install -q --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .

/content
fatal: destination path '/content/apex' already exists and is not an empty directory.
/content/apex
HEAD is now at 37cdaf4 fixing batchnorm 1d input (#590)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for apex
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for apex
ERROR: Could not bu

In [10]:
%cd /content/glow-tts/monotonic_align
!python setup.py build_ext --inplace

/content/glow-tts/monotonic_align
error: could not create 'monotonic_align/core.cpython-310-x86_64-linux-gnu.so': No such file or directory


## 4. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 구글 드라이브에서 가져옵니다.

In [11]:
%cd /content/glow-tts
# !gdown --id 1UpoBaZRTJXkTdsoemLBWV48QClm6hpTX -O filelists.zip
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

/content/glow-tts


## 5. 사전 학습 데이터 불러오기

사전 학습 데이터가 구글 드라이브에 존재하지 않을 경우,  
SleepingCE Speech Dataset의 사전 학습 데이터를 내려받습니다.

In [12]:
%cd /content/glow-tts
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/glow-tts"
if not Path("/content/drive/My Drive/Colab Notebooks/data/glow-tts/config.json").exists():
    !gdown --id 1js-v4ZjeOEPO1XpcBHFfwk7Ta8aDBAkg -O public.tar.gz
    !tar -zxf public.tar.gz -C "/content"
    !cp -R /content/public/glow-tts/* "/content/drive/My Drive/Colab Notebooks/data/glow-tts/"

/content/glow-tts


## 6. TensorBoard 실행

학습 진행을 확인하기 위해 TensorBoard를 실행합니다.

최신 학습 진행 상황을 확인하려면 TensorBoard 우측 상단의 새로고침 아이콘을 클릭해주세요.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/glow-tts"

Launching TensorBoard...

## 7. Glow-TTS 학습 진행

실제 Glow-TTS 학습을 진행합니다.

학습이 정상적으로 진행되면, 이 셀은 종료되지 않고 계속 실행되는 상태를 유지합니다.

학습 진행 정도를 시각적으로 확인하시려면 바로 위의 TensorBoard를 참고해주세요.

학습된 결과를 확인하고 싶으시다면, 가장 최근 체크포인트를 [SCE-TTS: 음성합성 데모](https://colab.research.google.com/drive/13pqat2mWsMha7Vn_-Q5_Ih8MDkvz3q5a)에서 불러와 확인할 수 있습니다.

In [ ]:
%cd /content/glow-tts
!python init.py -c configs/base.json -m "/content/drive/My Drive/Colab Notebooks/data/glow-tts"
!python train.py -c configs/base.json -m "/content/drive/My Drive/Colab Notebooks/data/glow-tts"